In [1]:
import findspark
findspark.init()

In [1]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql import Row
import pyspark.sql.functions as f

import pyspark.sql.types as t

import pyspark.ml.feature as feat

import numpy as np
import pandas as pd

# Pipeline
from pyspark.ml import Pipeline, PipelineModel

# Machine learning
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# About Dataset
Use the information:
-  product ID, 
-  User ID, 
- Rating

in dataset Shopee: Products_ThoiTrangNam build a recommended sysytem

**Then make recommendations to some specific users**

In [2]:
SparkContext.setSystemProperty(key='spark.hadoop.dfs.client.use.datanode.hostname',value='true')

In [3]:
sc =SparkContext()

In [4]:
sc.setLogLevel("ERROR")

In [5]:
spark = SparkSession(sc)

In [6]:
df = spark.read.csv(path='../DATA/clean_details.csv',inferSchema=True, header=True)

In [7]:
#sc.cancelAllJobs()

# Overview

In [8]:
df.show(10)
df.printSchema()
print(f'There are {df.count()} rows and {len(df.columns)} columns in dataframe')

+----------+-------+------+
|product_id|user_id|rating|
+----------+-------+------+
|       190|      1|     5|
|       190|      2|     5|
|       190|      3|     5|
|       190|      4|     5|
|       190|      5|     5|
|       190|      6|     5|
|       190|      7|     5|
|       190|      8|     5|
|       190|      9|     5|
|       190|     10|     5|
+----------+-------+------+
only showing top 10 rows

root
 |-- product_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)

There are 1024482 rows and 3 columns in dataframe


## check null

In [9]:
df.select([f.count(f.when(f.isnan(c), c)).alias(c) for c in df.columns]).show()

+----------+-------+------+
|product_id|user_id|rating|
+----------+-------+------+
|         0|      0|     0|
+----------+-------+------+



In [10]:
df.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in df.columns]).show()

+----------+-------+------+
|product_id|user_id|rating|
+----------+-------+------+
|         0|      0|     0|
+----------+-------+------+



## Descirble data

In [11]:
cat_col = df.columns
cat_col

['product_id', 'user_id', 'rating']

## Category data

### Check distinct values

In [12]:
df.select([f.count_distinct(c).alias(c) for c in df.columns]).show()

+----------+-------+------+
|product_id|user_id|rating|
+----------+-------+------+
|     31267| 650636|     5|
+----------+-------+------+



### Check top5 disticnt value count of each category columns in df 

In [13]:
for col in cat_col:
    (df.groupby(col).count()
     .withColumn('Normalize',(f.col('count')/df.count()))
     .orderBy(f.col('count').desc())
     .show(5,vertical=False,truncate=False))

+----------+-----+---------------------+
|product_id|count|Normalize            |
+----------+-----+---------------------+
|1731      |412  |4.021544546414676E-4 |
|177       |395  |3.8556070287228085E-4|
|231       |391  |3.816562906912957E-4 |
|17194     |389  |3.797040846008031E-4 |
|11131     |387  |3.7775187851031057E-4|
+----------+-----+---------------------+
only showing top 5 rows

+-------+-----+---------------------+
|user_id|count|Normalize            |
+-------+-----+---------------------+
|199    |19615|0.019146261232505794 |
|159    |2585 |0.002523226371961635 |
|831    |2541 |0.002480277837970799 |
|324    |2415 |0.0023572888542697677|
|860    |2088 |0.0020381031584742336|
+-------+-----+---------------------+
only showing top 5 rows

+------+------+--------------------+
|rating|count |Normalize           |
+------+------+--------------------+
|5     |777662|0.759078246372313   |
|4     |118212|0.11538709318465332 |
|3     |63051 |0.061544273105823236|
|1     |41447 |0.

In [14]:
df = df.withColumnRenamed(existing='user_id',new='usr_id')

# Preprocessing

In [15]:
df.show(10)

+----------+------+------+
|product_id|usr_id|rating|
+----------+------+------+
|       190|     1|     5|
|       190|     2|     5|
|       190|     3|     5|
|       190|     4|     5|
|       190|     5|     5|
|       190|     6|     5|
|       190|     7|     5|
|       190|     8|     5|
|       190|     9|     5|
|       190|    10|     5|
+----------+------+------+
only showing top 10 rows



In [16]:
strindex = feat.StringIndexer(inputCols=['usr_id','product_id'],outputCols=['user_id','prd_id']).fit(df)

In [17]:
final_df = strindex.transform(df)

In [18]:
final_df.show(10)
final_df.printSchema()

+----------+------+------+--------+------+
|product_id|usr_id|rating| user_id|prd_id|
+----------+------+------+--------+------+
|       190|     1|     5|113654.0| 367.0|
|       190|     2|     5| 50699.0| 367.0|
|       190|     3|     5|284299.0| 367.0|
|       190|     4|     5|376354.0| 367.0|
|       190|     5|     5|  6462.0| 367.0|
|       190|     6|     5|  6657.0| 367.0|
|       190|     7|     5|625739.0| 367.0|
|       190|     8|     5|   477.0| 367.0|
|       190|     9|     5| 32734.0| 367.0|
|       190|    10|     5| 33207.0| 367.0|
+----------+------+------+--------+------+
only showing top 10 rows

root
 |-- product_id: integer (nullable = true)
 |-- usr_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- user_id: double (nullable = false)
 |-- prd_id: double (nullable = false)



In [19]:
final_df.select([f.count(f.when(f.isnan(c), c)).alias(c) for c in final_df.columns]).show()

+----------+------+------+-------+------+
|product_id|usr_id|rating|user_id|prd_id|
+----------+------+------+-------+------+
|         0|     0|     0|      0|     0|
+----------+------+------+-------+------+



In [20]:
final_df.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in final_df.columns]).show()

+----------+------+------+-------+------+
|product_id|usr_id|rating|user_id|prd_id|
+----------+------+------+-------+------+
|         0|     0|     0|      0|     0|
+----------+------+------+-------+------+



# Split Data

In [21]:
train, test = final_df.randomSplit([0.8,0.2],seed=42)

# Model

# Evaluation

In [22]:
als = ALS(maxIter=10,
          regParam=0.01, 
          userCol='user_id',
          itemCol='prd_id', 
          ratingCol='rating', 
          coldStartStrategy='drop',
          rank=10,
          nonnegative=True)
model_t = als.fit(train)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\env39\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\ProgramData\Anaconda3\envs\env39\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\env39\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\ProgramData\Anaconda3\envs\env39\lib\site-packages\py4j\clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [49]:
# evaluate 
pred_t = model_t.transform(test)
evaluator = RegressionEvaluator(metricName='rmse',
                                labelCol='rating',
                                predictionCol='prediction')
rmse = evaluator.evaluate(pred_t)
print("Root-mean square error = "+str(rmse))

Root-mean square error = 1.3074620224307896


In [50]:
user_recs = model_t.recommendForAllUsers(10)

In [51]:
user_recs.printSchema()

root
 |-- user_id: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- prd_id: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [64]:
user_recs.show(10)

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|     31|[{2608, 6.0681133...|
|     34|[{11808, 8.067472...|
|     53|[{9466, 7.335936}...|
|     65|[{10928, 7.626090...|
|     78|[{6875, 8.079903}...|
|     85|[{2700, 8.284455}...|
|    108|[{10731, 7.793638...|
|    133|[{10928, 8.332048...|
|    137|[{5377, 8.048038}...|
|    148|[{10928, 9.205428...|
+-------+--------------------+
only showing top 10 rows



In [65]:
result = user_recs.select(f.col('user_id'),
                       f.explode(f.col('recommendations')))

result = (result.withColumn('prd_id', 
                               result.col.getField('prd_id'))
             .withColumn('rating',  
                         result.col.getField('rating'))
            )
result.show()

+-------+------------------+------+---------+
|user_id|               col|prd_id|   rating|
+-------+------------------+------+---------+
|     31| {2608, 6.0681133}|  2608|6.0681133|
|     31| {2941, 6.0042872}|  2941|6.0042872|
|     31| {11802, 5.924115}| 11802| 5.924115|
|     31|  {7332, 5.847309}|  7332| 5.847309|
|     31| {7524, 5.8088546}|  7524|5.8088546|
|     31|{11395, 5.7949514}| 11395|5.7949514|
|     31|  {10731, 5.70828}| 10731|  5.70828|
|     31|  {5389, 5.664432}|  5389| 5.664432|
|     31|  {9703, 5.643629}|  9703| 5.643629|
|     31| {4395, 5.6272407}|  4395|5.6272407|
|     34| {11808, 8.067472}| 11808| 8.067472|
|     34|  {6875, 7.980544}|  6875| 7.980544|
|     34|{11802, 7.3937116}| 11802|7.3937116|
|     34| {9866, 7.2814918}|  9866|7.2814918|
|     34|  {4961, 7.113665}|  4961| 7.113665|
|     34| {7769, 7.0938873}|  7769|7.0938873|
|     34| {9425, 7.0872297}|  9425|7.0872297|
|     34|  {6022, 7.034169}|  6022| 7.034169|
|     34|  {3474, 7.011326}|  3474

In [66]:
result.select('rating').describe().show()

+-------+-----------------+
|summary|           rating|
+-------+-----------------+
|  count|           194080|
|   mean|7.053887954934731|
| stddev|0.972378526601487|
|    min|        1.0987473|
|    max|         17.95811|
+-------+-----------------+



# Check new data

In [69]:
final_df.show(5)

+--------------+----------+-------+-------+------+
|    reviewerID|      asin|overall|user_id|prd_id|
+--------------+----------+-------+-------+------+
|A1VXOAVRGKGEAK|0439893577|    5.0|14349.0|2524.0|
| A8R62G708TSCM|0439893577|    4.0|18115.0|2524.0|
|A21KH420DK0ICA|0439893577|    5.0| 4454.0|2524.0|
| AR29QK6HPFYZ4|0439893577|    5.0|18990.0|2524.0|
| ACCH8EOML6FN5|0439893577|    4.0| 2769.0|2524.0|
+--------------+----------+-------+-------+------+
only showing top 5 rows



In [81]:
check_reviewer = ['A3GJPLCZCDXXG6', 'A34U85WY8ZWBPV', 'A2VIY2TL6QPYLG']

- test if there are more than 2 values for one reviewer 

In [79]:
check_distinct = final_df.select('reviewerID','user_id').distinct().groupby('reviewerID').agg(f.count(f.col('user_id')).alias('count_id'))
check_distinct.show(10)

+--------------+--------+
|    reviewerID|count_id|
+--------------+--------+
| ADF0URBXWJQOJ|       1|
| ARIFCL50JD5SK|       1|
| ATRQPL5OG4NIP|       1|
|A1OPV6BD37T1VL|       1|
| A2S0HZ2WHXRKD|       1|
|A2P6QCZWW3H1X6|       1|
| A9XCW04U6X4NP|       1|
|A3LUUDFQJWLRTB|       1|
| ATHUV6LFX2AXM|       1|
|A3OPXNND84JPV6|       1|
+--------------+--------+
only showing top 10 rows



In [80]:
check_distinct.filter(f.col('count_id')>1).show()

+----------+--------+
|reviewerID|count_id|
+----------+--------+
+----------+--------+



In [83]:
map_data = final_df.select('reviewerID','user_id').distinct().filter(f.col('reviewerID').isin(check_reviewer))
map_data.show()

+--------------+-------+
|    reviewerID|user_id|
+--------------+-------+
|A34U85WY8ZWBPV|16527.0|
|A2VIY2TL6QPYLG| 7861.0|
|A3GJPLCZCDXXG6| 6190.0|
+--------------+-------+



In [87]:
id_reviewer_encode = [i[0] for i in map_data.select('user_id').collect()]
id_reviewer_encode

[16527.0, 7861.0, 6190.0]

- List of product id encoder is recommended by system  

In [88]:
result.filter(f.col('user_id').isin(id_reviewer_encode)).show()

+-------+------------------+------+---------+
|user_id|               col|prd_id|   rating|
+-------+------------------+------+---------+
|   7861| {6874, 7.1505904}|  6874|7.1505904|
|   7861|  {5309, 7.061749}|  5309| 7.061749|
|   7861| {5308, 6.8394947}|  5308|6.8394947|
|   7861|  {3500, 6.692243}|  3500| 6.692243|
|   7861| {3503, 6.6303864}|  3503|6.6303864|
|   7861| {3209, 6.6245437}|  3209|6.6245437|
|   7861| {11883, 6.582908}| 11883| 6.582908|
|   7861| {6794, 6.5016327}|  6794|6.5016327|
|   7861|  {7944, 6.410063}|  7944| 6.410063|
|   7861| {3099, 6.4076023}|  3099|6.4076023|
|   6190| {5377, 7.2214284}|  5377|7.2214284|
|   6190| {8154, 6.9718266}|  8154|6.9718266|
|   6190| {2649, 6.8427224}|  2649|6.8427224|
|   6190| {10731, 6.788498}| 10731| 6.788498|
|   6190|  {8158, 6.787895}|  8158| 6.787895|
|   6190|  {6095, 6.750331}|  6095| 6.750331|
|   6190|  {9424, 6.742305}|  9424| 6.742305|
|   6190|{11832, 6.7392454}| 11832|6.7392454|
|   6190|  {6708, 6.711317}|  6708